# Read Data from Silver VAUsage Table and Write to Gold VAUsage Action Counts Table
To run this notebook, import it into Azure Synapse and attach it to an Apache Spark Pool.
Choose the "Small" Node Size, and choose "3" as the Number of Nodes.
Be sure to run the "rate-streaming-to-bronze" Notebook and the "bronze-to-silver-vausage" Notebook beforehand.

In [ ]:
%%spark
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

## Configure the Storage Account (to read from)
Replace the value `<storageAccountName>` with the name of the storage account where the Silver Telemetry Table data is stored.

In [ ]:
%%spark
val storageAccountName = "<storageAccountName>"
val silverDataLocation: String = "abfss://datalake@"+storageAccountName+".dfs.core.windows.net/silverSynapse/VAUsage"
val goldDataLocation : String = "abfss://datalake@"+storageAccountName+".dfs.core.windows.net/goldSynapse/VAUsageActionCounts"

## Read the Data
Here the data is read from the `silverDataLocation` specified in the previous cell, which is configured using the value inputted for `storageAccount`.

In [ ]:
%%spark
var silverDF = spark.readStream.format("delta").load(silverDataLocation)

## Configure the Schema of the Data
The schema of the Dataframe is configured to match the schema of the Gold VAUSage Action Counts Table.

In [ ]:
%%spark
var goldDF = silverDF.withWatermark("ProcessedTimestamp", "10 second").groupBy(
    window(col("ProcessedTimestamp"), "10 seconds", "10 second"),
    col("Object"),
    col("Action")
  ).count().withColumn("WindowStartDate", to_date(col("window").getItem("start")))

goldDF.printSchema()

## Write Data to Gold VAUSage Action Counts Table

In [ ]:
%%spark
val goldVAUsageQuery = goldDF.writeStream.format("delta").
outputMode("append").
partitionBy("WindowStartDate").
option("checkpointLocation", goldDataLocation + "/checkpoint").
start(goldDataLocation) 

## View the Data

In [ ]:
%%spark
var goldViewDF = spark.read.format("delta").load(goldDataLocation)
goldViewDF.orderBy(col("window.start").desc).show()

In [ ]:
%%spark
goldViewDF.printSchema()

In [ ]:
%%spark
display(goldViewDF.orderBy(col("window.start").desc))